In [1]:
import torch
from torch.utils.data import Dataset
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

batch_len = 4

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=torchvision.transforms.Compose([torchvision.transforms.Resize(32),
                         torchvision.transforms.ToTensor()])
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=torchvision.transforms.Compose([torchvision.transforms.Resize(32),
                         torchvision.transforms.ToTensor()])
)

training_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_len, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_len, shuffle=False, num_workers=2)

100%|██████████| 9912422/9912422 [00:00<00:00, 80954479.82it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 94637260.80it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 20589750.80it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 15718258.06it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [2]:
print(len(training_data),len(training_loader))

60000 15000


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # nn.Conv2d(input image channel, output channels, square convolution)
        # kernel
        self.conv1 = nn.Conv2d(1, 64, 3, stride=1, padding=1)
        self.conv1_bn = nn.BatchNorm2d(64)

        self.conv2 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.conv2_bn = nn.BatchNorm2d(128)

        self.conv3 = nn.Conv2d(128, 256, 3, stride=1, padding=1)
        self.conv3_bn = nn.BatchNorm2d(256)

        self.conv4 = nn.Conv2d(256, 256, 3, stride=1, padding=1)
        self.conv4_bn = nn.BatchNorm2d(256)

        self.conv5 = nn.Conv2d(256, 512, 3, stride=1, padding=1)
        self.conv5_bn = nn.BatchNorm2d(512)

        self.conv6 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.conv6_bn = nn.BatchNorm2d(512)

        self.conv7 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.conv7_bn = nn.BatchNorm2d(512)

        self.conv8 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.conv8_bn = nn.BatchNorm2d(512)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(512, 4096)
        self.fc1_dp = nn.Dropout(p=0.5)

        self.fc2 = nn.Linear(4096, 4096)
        self.fc2_dp = nn.Dropout(p=0.5)

        self.fc3 = nn.Linear(4096, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1_bn(self.conv1(x))), 2)
        x = F.max_pool2d(F.relu(self.conv2_bn(self.conv2(x))), 2)
        x = F.relu(self.conv3_bn(self.conv3(x)))
        x = F.max_pool2d(F.relu(self.conv4_bn(self.conv4(x))), 2)
        x = F.relu(self.conv5_bn(self.conv5(x)))
        x = F.max_pool2d(F.relu(self.conv6_bn(self.conv6(x))), 2)
        x = F.relu(self.conv7_bn(self.conv7(x)))
        x = F.max_pool2d(F.relu(self.conv8_bn(self.conv8(x))), 2)

        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension

        x = self.fc1_dp(F.relu(self.fc1(x)))
        x = self.fc2_dp(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

In [4]:
model_E1 = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_E1.parameters(), lr=0.01)

In [5]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.
    total_loss = 0.
    train_correct = 0

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        train_inputs, train_labels = data
        print(train_inputs.shape)
        break

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        train_outputs = model_E1(train_inputs)

        # Computer Correctness
        train_predict = train_outputs.argmax(dim=1)
        train_correct += torch.eq(train_predict, train_labels).sum().item()

        # Compute the loss and its gradients
        train_loss = criterion(train_outputs, train_labels)
        train_loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += train_loss.item()
        total_loss += train_loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print("batch {} loss: {}".format(i + 1, last_loss))
            print("total correct: {}, total accuracy: {}".format(train_correct, train_correct/(batch_len * (i + 1))))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return total_loss/(i + 1), train_correct/(batch_len * (i + 1))

In [ ]:
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/MNIST_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

train_losses = []
test_losses = []
train_accuracys = []
test_accuracys = []

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model_E1.train(True)
    avg_train_loss, train_accuracy = train_one_epoch(epoch_number, writer)
    train_losses.append(avg_train_loss)
    train_accuracys.append(train_accuracy)

    # We don't need gradients on to do reporting
    model_E1.train(False)

    test_total_loss = 0.0
    test_correct = 0
    for i, vdata in enumerate(test_loader):
        test_inputs, test_labels = vdata
        test_outputs = model_E1(test_inputs)

        # test loss
        test_loss = criterion(test_outputs, test_labels)
        test_total_loss += test_loss.item()

        # test correctness
        test_predict = test_outputs.argmax(dim=1)
        test_correct += torch.eq(test_predict, test_labels).sum().float().item()

    avg_test_loss = test_total_loss/(i + 1)
    test_accuracy = test_correct/(batch_len * (i + 1))
    test_losses.append(avg_test_loss)
    test_accuracys.append(test_accuracy)
    print("Train Loss: {}, Test Loss: {}".format(avg_train_loss, avg_test_loss))
    print("Train Accuracy: {}, Test Accuracy: {}".format(train_accuracy, test_accuracy))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars("Training vs. Validation Loss", {"Training": avg_test_loss, "Validation": avg_test_loss}, epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_test_loss < best_vloss:
        best_vloss = avg_test_loss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model_E1.state_dict(), model_path)

    epoch_number += 1

writer.close()

EPOCH 1:
torch.Size([4, 1, 32, 32])
Train Loss: 0.0, Test Loss: 2.3025168307304384
Train Accuracy: 0.0, Test Accuracy: 0.1135
EPOCH 2:
torch.Size([4, 1, 32, 32])
Train Loss: 0.0, Test Loss: 2.3025168307304384
Train Accuracy: 0.0, Test Accuracy: 0.1135
EPOCH 3:
torch.Size([4, 1, 32, 32])


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.title("Training Loss")
plt.plot(range(1,EPOCHS+1), train_losses)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Test Loss")
plt.plot(range(1,EPOCHS+1), test_losses)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Training Accuracy")
plt.plot(range(1,EPOCHS+1), train_accuracys)
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Test Accuracy")
plt.plot(range(1,EPOCHS+1), test_accuracys)
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
Hflip_test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=torchvision.transforms.Compose([torchvision.transforms.Resize(32),torchvision.transforms.RandomHorizontalFlip(p=1),torchvision.transforms.ToTensor()])
)
Vflip_test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=torchvision.transforms.Compose([torchvision.transforms.Resize(32),torchvision.transforms.RandomVerticalFlip(p=1),torchvision.transforms.ToTensor()])
)

Hflip_test_loader = torch.utils.data.DataLoader(Hflip_test_data, batch_size=batch_len, shuffle=False, num_workers=2)
Vflip_test_loader = torch.utils.data.DataLoader(Vflip_test_data, batch_size=batch_len, shuffle=False, num_workers=2)

In [ ]:
Hflip_test_correct = 0
Vflip_test_correct = 0

# We don't need gradients on to do reporting
model_E1.train(False)

for i, vdata in enumerate(Hflip_test_loader):
    Hflip_test_inputs, Hflip_test_labels = vdata
    Hflip_test_outputs = model_E1(Hflip_test_inputs)

    # test correctness
    Hflip_test_predict = Hflip_test_outputs.argmax(dim=1)
    Hflip_test_correct += torch.eq(Hflip_test_predict, Hflip_test_labels).sum().float().item()

Hflip_test_accuracy = Hflip_test_correct/(batch_len * (i + 1))

for j, vdata in enumerate(Vflip_test_loader):
    Vflip_test_inputs, Vflip_test_labels = vdata
    Vflip_test_outputs = model_E1(Vflip_test_inputs)

    # test correctness
    Vflip_test_predict = Vflip_test_outputs.argmax(dim=1)
    Vflip_test_correct += torch.eq(Vflip_test_predict, Vflip_test_labels).sum().float().item()

Hflip_test_accuracy = Hflip_test_correct/(batch_len * (i + 1))
Vflip_test_accuracy = Vflip_test_correct/(batch_len * (j + 1))

In [ ]:
print("Test Accuracy For Random Horizontal Flip: {}".format(Hflip_test_accuracy))
print("Test Accuracy For Random Vertical Flip: {}".format(Vflip_test_accuracy))

In [ ]:
flip_name = ("Horizontal Flip", "Vertical Flip")
flip_accuracy = (Hflip_test_accuracy, Vflip_test_accuracy)
plt.figure(figsize=(5,5))
plt.bar(flip_name, flip_accuracy, width = 0.4)
plt.xlabel("Flip Type")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
Var001_test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=torchvision.transforms.Compose([torchvision.transforms.Resize(32),torchvision.transforms.ToTensor(),torchvision.transforms.Lambda(lambda x : x + 0.1 * torch.randn_like(x))])
)
Var01_test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=torchvision.transforms.Compose([torchvision.transforms.Resize(32),torchvision.transforms.ToTensor(),torchvision.transforms.Lambda(lambda x : x + torch.sqrt(0.1) * torch.randn_like(x))])
)
Var1_test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=torchvision.transforms.Compose([torchvision.transforms.Resize(32),torchvision.transforms.ToTensor(),torchvision.transforms.Lambda(lambda x : x + torch.randn_like(x))])
)

Var001_test_loader = torch.utils.data.DataLoader(Var001_test_data, batch_size=batch_len, shuffle=False, num_workers=2)
Var01_test_loader = torch.utils.data.DataLoader(Var01_test_data, batch_size=batch_len, shuffle=False, num_workers=2)
Var1_test_loader = torch.utils.data.DataLoader(Var1_test_data, batch_size=batch_len, shuffle=False, num_workers=2)

In [ ]:
Var001_test_correct = 0
Var01_test_correct = 0
Var1_test_correct = 0

# We don't need gradients on to do reporting
model_E1.train(False)

for i, vdata in enumerate(Var001_test_loader):
    Var001_test_inputs, Var001_test_labels = vdata
    Var001_test_outputs = model_E1(Var001_test_inputs)

    # test correctness
    Var001_test_predict = Var001_test_outputs.argmax(dim=1)
    Var001_test_correct += torch.eq(Var001_test_predict, Var001_test_labels).sum().float().item()

for k, vdata in enumerate(Var1_test_loader):
    Var1_test_inputs, Var1_test_labels = vdata
    Var1_test_outputs = model_E1(Var1_test_inputs)

    # test correctness
    Var1_test_predict = Var1_test_outputs.argmax(dim=1)
    Var1_test_correct += torch.eq(Var1_test_predict, Var1_test_labels).sum().float().item()

for j, vdata in enumerate(Var01_test_loader):
    Var01_test_inputs, Var01_test_labels = vdata
    Var01_test_outputs = model_E1(Var01_test_inputs)

    # test correctness
    Var01_test_predict = Var01_test_outputs.argmax(dim=1)
    Var01_test_correct += torch.eq(Var01_test_predict, Var01_test_labels).sum().float().item()

Var001_test_accuracy = Var001_test_correct/(batch_len * (i + 1))
Var01_test_accuracy = Var01_test_correct/(batch_len * (j + 1))
Var1_test_accuracy = Var1_test_correct/(batch_len * (k + 1))